In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from fitz import Rect

import numpy as np


from pdf_scraper.doc_utils import open_exam, get_images, filter_point_images
from pdf_scraper.doc_utils import get_doc_line_df
from pdf_scraper.doc_utils import get_captions
from pdf_scraper.block_utils import clean_blocks
from pdf_scraper.line_utils import print_line_table, get_line_df

In [3]:
from PIL import Image
from io import BytesIO

def show_image(image):
    img_bytes = image["image"]
    img_stream = BytesIO(img_bytes)
    img = Image.open(img_stream)
    display(img)

1. Get all images.
   - filter image artifacts and stitch stripped images. 
2. Get all text lines.
3. Identify and Remove captions from text lines.
4. Identify and resort dual column text.
5. Identify and remove page headers and footers. 

In [4]:
# 2001 - good
# 2002 - non-caption close to image counted as caption. ==> to count as inside we say it must be > 0.2 inside.
# 2003 - good
# 2004 - good
# 2005 - 2 captions, captured but pictures are not all together, they are partitioned in boxes.
# 2006 - good -> point artifacts filtered
# 2007 - good
# 2008 - good
# 2009 - good
# 2010 - good
# 2011 - Captions below images page 2 and 3.
# 2012 - Mostly good, then some captions below image.
# 2013 - page 3 and 2, image cut into slices, captions overlap with many. page 4 caption below image.
# 2014 - caption below image page 3. all others good
# 2015 - 2018 good
# 2019 - caption not contained in photo.
# 2020 - good
# 2022 - good
# 2023 - good
# 2024 - good
# 2025 - good
# For caption prediction, we should have distance to nearest image
# distance to nearest text.
year=2013
doc = open_exam(year,"english","al",1)
doc_df = get_doc_line_df(doc)
images = get_images(doc)
print(f"number of raw images                : {len(images):10}")
images = filter_point_images(images)
print(f"number of images after  point filter: {len(images):10}")
images = get_captions(doc_df, images)

number of raw images                :        260
number of images after  point filter:        260
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
        text  page
42  IMAGE 1      2
         text  page
133  IMAGE 2      3


In [6]:
def bbox_distance(bbox1, bbox2):
    """
    Calculates the minimum edge-to-edge distance between two bounding boxes.
    Each box is in [x_min, y_min, x_max, y_max] format.
    Returns 0 if they overlap or touch.
    """
    x_min1, y_min1, x_max1, y_max1 = bbox1
    x_min2, y_min2, x_max2, y_max2 = bbox2

    dx = max(x_min2 - x_max1, x_min1 - x_max2, 0)
    dy = max(y_min2 - y_max1, y_min1 - y_max2, 0)

    return np.hypot(dx, dy)

def closest_image(bbox, images, n_page):
    dist = 100000
    page_images = [img for img in images if img["page"]==n_page]
    for image in page_images:
        if (bbox_distance(bbox,image["bbox"])) < dist:
            closest=image
    return image

In [89]:
display(doc_df[doc_df.text=='Warstones\xa0Library\xa0'][["x0","y0","x1","y1"]])

bbox = tuple(doc_df[doc_df.text=='Warstones\xa0Library\xa0'][["x0","y0","x1","y1"]].values[0])
img = closest_image(bbox,images)
img["caption"]='Warstones\xa0Library\xa0'

,x0,y0,x1,y1


IndexError: index 0 is out of bounds for axis 0 with size 0

In [83]:
page_2_strips = [img for img in images if img["page"]==2]
print(len(images)-len(page_2_strips))
print(len(page_2_strips))
img=page_2_strips[0]
x0,y0,x1,y1 = img["bbox"]
print(x1-x0)
print(y1-y0)
threshold=5
(x1 - x0) < threshold and (y1 - y0) < threshold

6
254
198.17999267578125
0.779998779296875


False

In [8]:
page_2_strips = [image for image in images if image["page"]==2  ]
print(len(page_2_strips))
heights=[strip["height"] for strip in page_2_strips ]
widths =[strip["width"] for strip in page_2_strips ]
display(np.unique(heights))
display(np.unique(widths))

254


array([1])

array([367])

In [9]:
def is_horizontal_strip(img):
    return img["height"] <2 and img["width"] > 40

all([is_horizontal_strip(img) for img in page_2_strips] )

True

In [ ]:
y0s = [img["bbox"][1] for img in page_2_strips]

for i in range(len(y0s[:-2]) ):
    if y0s[i+1] < y0s[i]:
        print("not monotonic!")

In [18]:
doc[1].get_text("dict")["width"]/8

74.40249633789062

In [28]:
page_2_images[0]["caption"].isspace() or not page_2_images[0]["caption"]

True

In [ ]:
blocks=[]
images = []
dfs = []
for i, page in enumerate(doc):
    page_blocks  = page.get_text("dict",sort=True)["blocks"]

    text_blocks  = [block for block in page_blocks if not block["type"]]
    text_blocks = clean_blocks(text_blocks)

    image_blocks = [block for block in page_blocks if     block["type"]]
    for image_block in image_blocks:
        image_block["page"]= i+1

    page_lines   = [ line for block in text_blocks for line in block["lines"]]
    page_df = get_line_df(page_lines)
    page_df["page"] = i+1
    dfs.append(page_df)

grand_df = pd.concat(dfs,ignore_index=True)
grand_df["dual_col"]=0

# label dual column text

## Label page 2

In [16]:
grand_df[grand_df.page==2].text.head(15)

24    SECTION I                       COMPREHENDING ...
25                        TEXT 1 – FROM GENRE to GENRE 
26    This text consists of two elements: firstly, e...
27    The Misadventures of a Dithering Writer in Thi...
28    writing in different genres.  The second eleme...
29    I flit anxiously and eagerly from genre to gen...
30    yourself be led by the child that you were.  T...
31     I always have a few stories on the go.  Some of 
32       is a tendency I adhered to upon my resumption 
33     and, indeed, return to when it all threatens to 
34     them are like eels – they slip away if I do not 
35    make a fast grab.  Some are like bold children – 
36                                   get away from me. 
37      they pay absolutely no attention to anything I 
38     tell them to do.  One or two arrive unannounced 
Name: text, dtype: object

function
- takes in 4 lines: top left, top right, bottom left, bottom right
- makes copy of data frame
- identifies all lines between these. (max and min index)
- sorts this part of the data frame 
- returns tuple (sorted data frame, indices)

Then you can reassign this sorted data frame to the original data frame between
the provided indices.

Or else it can change the passed data frame in place which would be as useufl. 

In [17]:
def setDualCols(grand_df: pd.DataFrame, page_num:int, bookends: tuple[str]):
    page_df = grand_df[grand_df.page==page_num].copy()
    l1, r1, l2, r2 = bookends
    for line in bookends:
        print(page_df[page_df.text.str.contains(line)].index)
    indices = [page_df[page_df.text.str.contains(line)].index for line in bookends]
    top    = min(indices).values[0]
    bottom = max(indices).values[0]
    dual_cols = page_df[top:bottom+1].copy()
    dual_cols.sort_values(["x0","y0"],inplace=True)
    dual_cols["daul_col"]=1

    grand_df.loc[top:bottom+1] = dual_cols

    return grand_df


In [18]:
grand_df[grand_df.page==2].text.head(40)

24    SECTION I                       COMPREHENDING ...
25                        TEXT 1 – FROM GENRE to GENRE 
26    This text consists of two elements: firstly, e...
27    The Misadventures of a Dithering Writer in Thi...
28    writing in different genres.  The second eleme...
29    I flit anxiously and eagerly from genre to gen...
30    yourself be led by the child that you were.  T...
31     I always have a few stories on the go.  Some of 
32       is a tendency I adhered to upon my resumption 
33     and, indeed, return to when it all threatens to 
34     them are like eels – they slip away if I do not 
35    make a fast grab.  Some are like bold children – 
36                                   get away from me. 
37      they pay absolutely no attention to anything I 
38     tell them to do.  One or two arrive unannounced 
39      I am, at various times, a reluctant, plodding, 
40    instinctive, spontaneous writer.  At times I f...
41        from the farthest recesses of my imagi

In [19]:
line_l1 = "I flit anxious"
line_r1 = "yourself be led by the child"
line_lf = "imagination had not abandoned me"
line_rf = "lose almost all the time"

bookends = [line_l1,line_r1, line_lf, line_rf]

grand_df = setDualCols(grand_df, 2, bookends)

Index([29], dtype='int64')
Index([30], dtype='int64')
Index([106], dtype='int64')
Index([107], dtype='int64')


In [20]:
grand_df[grand_df.page==2].text.head(40)

24    SECTION I                       COMPREHENDING ...
25                        TEXT 1 – FROM GENRE to GENRE 
26    This text consists of two elements: firstly, e...
27    The Misadventures of a Dithering Writer in Thi...
28    writing in different genres.  The second eleme...
53                              immediate attention.   
54             I don’t know if my writing is in anyway 
55        I have started several novels.  There is the 
56    distinctive.  I am an aural learner as opposed...
57       edgy-existential one about the brother-sister 
58        assassination squad.  There is the comedy-of-
59            say the more common visual learning that 
60         attends so much writing.  I can hear things 
61      desperation one about the office slave finally 
62           tipped over the edge by a boss constantly 
63      before I see them.  My reasons for writing are 
64    partly intrinsic, partly spiritual, partly fan...
65       referred to as the highly evolved veget

In [75]:
page2_df = grand_df[grand_df.page==2].copy()
line_l1 = "I flit anxious"
line_r1 = "yourself be led by the child"

line_lf = "imagination had not abandoned me"
line_rf = "lose almost all the time"

bookends = [line_l1,line_r1, line_lf, line_rf]
for line in bookends:
    print(page2_df[page2_df.text.str.contains(line)].index)
indices = [page2_df[page2_df.text.str.contains(line)].index for line in bookends]
top = min(indices)
bottom = max(indices)

Index([29], dtype='int64')
Index([69], dtype='int64')
Index([68], dtype='int64')
Index([107], dtype='int64')


In [73]:
page2_text = page2_df[top.values[0]:bottom.values[0]+1].sort_values(["x0","y0"])
page2_text["dual_col"]=1

# Fixing stripped images

We were workin gthrough these functions from chat gpt:

In [ ]:
x0s = np.unique([strip["bbox"][0] for strip in page_2_strips])
n_strip_images = len(x0s)
for x0 in x0s:
    print(x0)
    strips = [strip for strip in page_2_strips if strip["bbox"][0]==x0]
len(strips)

343.44000244140625


254

In [14]:
page_2_strips[0]

{'number': 9,
 'type': 1,
 'bbox': (343.44000244140625,
  139.62002563476562,
  541.6199951171875,
  140.4000244140625),
 'width': 367,
 'height': 1,
 'ext': 'png',
 'colorspace': 1,
 'xres': 96,
 'yres': 96,
 'bpc': 8,
 'transform': (198.17999267578125,
  0.0,
  -0.0,
  0.7799999713897705,
  343.44000244140625,
  139.62002563476562),
 'size': 850,
 'image': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01o\x00\x00\x00\x01\x08\x02\x00\x00\x00\x9e\xd7\xb8\xdd\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00\x03\x04IDATx\x9c5\x92\xeb+\xe4a\x14\xc7\xf7? f\x18\x96q\x19E./\xdc\xd92\x841\xc4\xc8u\xdc\x19\xb2\xe6\r!\xb4V\xe1\x1d\xf1f\x0b\xf1b\xcb5\x11%V;\xb4\xc9-\xe6\xe22\xacFVS\xb4\xf8\x0f\xfc\x03\xfb\xe9\xfc\xda\xcf\x8b\xa7\xf3\x9c\xe7\xfb\x9c\xe7<\xe7\x9c\x0f}}}UUU\x99\x99\x99\x13\x13\x13\xbf\x84\xc1\xc1\xc1\x81\x81\x01\xab\xd5Z[[\xdb\xd1\xd11<<<99\x89svvvhh(\'\'\x87\xb5\xa5\xa5\x05Ayyyjj\xea\xd4\xd4\xd4\xd8\xd8XSS\xd3g\xa1\xa6\xa6&666##cuuudd\x84mLLLzzzii\xa9\xd

In [ ]:
from PIL import Image
import io

def is_horizontal_strip(img):
    return img["height"] <2 and img["width"] > 40

def get_stripped_images(images):
    stripped_images = []
    strips = [img for img in images if is_horizontal_strip(img)]
    x0s = np.unique([strip["bbox"][0] for strip in page_2_strips])
    if len(x0s) > 1:
        raise ValueError(
                f"Multiple stripped images detected on the page (x0s={x0s}). "
                "Refactor required to handle multiple horizontal strips."
            )
    return strips

def stitch_strips(image_blocks, output_path="stitched_image.png"):
    """
    Stitch a list of horizontal image strips (already sorted top-to-bottom) into a single image.
    """
    images = [Image.open(io.BytesIO(block["image"])) for block in image_blocks ]

    total_height = sum(img.height for img in images)
    max_width = max(img.width for img in images)

    stitched = Image.new("RGB", (max_width, total_height), (255, 255, 255))
    offset = 0
    for img in images:
        stitched.paste(img, (0, offset))
        offset += img.height

    stitched.save(output_path)

stitch_strips(page_2_strips)